In [1]:
import pandas as pd

In [2]:
filename = 'dataset1m.xlsx'# <- filename
df = pd.read_excel(filename)

In [3]:
# Flags
ANONIMIZE_JOBS = True
ANONIMIZE_ADDRESS = True
ANONIMIZE_SALARY = True
DELETE_RARE_ATTRIBUTES = True
SAVE_ANON_FILE = False

In [4]:
def salary_local_generalization(x):
    val = int(x)
    if val > 100000:
        return "Very high"
    elif val > 75000:
        return "High"
    elif val > 55000:
        return "Mid"
    else:
        return "Low"
def del_col():
    return '***'
def addr_mask(x):
    return x.split(" д. ")[0] + ' ***'
if ANONIMIZE_SALARY:
    df["Заработная плата"] = df["Заработная плата"].apply(salary_local_generalization)

# Прямой идентефикатор - всегда анонимизируем
df["ФИО работника"] = df["ФИО работника"].apply(del_col)


if ANONIMIZE_ADDRESS:
    df["Адрес работы"] = df["Адрес работы"].apply(addr_mask)

In [5]:
def func(x):
    s = str(x)
    if x == "Административный директор" or x == "Директор по маркетингу" or x == "Финансовый директор" or x == "Генеральный директор":
        return "Директор"
    elif x == "Водитель" or x == "Комендант" or x == "Охранник" or x == "Уборщик" or x == "Секретарь" or x == "Бухгалтер":
        return "Вспомогательный персонал"
    else:
        return s

if ANONIMIZE_JOBS:
    df["Должность"] = df["Должность"].apply(func)

In [6]:
def func(x):
    s = str(x)
    if s == "*":
        return s
    elif int(s[1:4]) in [929, 921, 931]:
        return "Megafon"
    elif int(s[1:4]) in [911, 981]:
        return "MTS"
    elif int(s[1:4]) in [961, 962, 963, 964, 903, 905, 906, 909, 960]:
        return "Beeline"
    elif int(s[1:4]) in [901, 952, 904, 950, 951]:
        return "Tele2"

# Прямой идентефикатор - всега анонимизируем
df["Номер телефона"] = df["Номер телефона"].apply(func)

In [7]:
def get_dict_and_k_anonimity():
    mydictionary = {}
    def getstrline(ind):
        return str(df["ФИО работника"].iloc[ind]) + str(df["Номер телефона"].iloc[ind]) + str(df["Адрес работы"].iloc[ind]) + str(df["Должность"].iloc[ind]) + str(df["Заработная плата"].iloc[ind])
    for row in range(df.shape[0]):
        s = getstrline(row)
        if s in mydictionary:
            mydictionary[s] += 1
        else:
            mydictionary[s] = 1
    return mydictionary, min(mydictionary.values())

In [8]:
mydict, k_anon = get_dict_and_k_anonimity()
print("K-anon before deletion = ", k_anon)

Current k-anon =  3


In [9]:
sorted_mydict = sorted(mydict.items(), key=lambda x:x[1])
for i in range(5):
    print(sorted_mydict[i])

('***BeelineРеки Смоленки наб.ПрограммистVery high', 3)
('***Beelineул. ВоронежскаяПрограммистHigh', 6)
('***Tele2ул. ВоронежскаяПрограммистVery high', 6)
('***Tele2Реки Смоленки наб.ПрограммистVery high', 7)
('***BeelineДмитровский пер.ПрограммистHigh', 8)


In [10]:
df.astype(str).values.flatten().tolist()
r = [''.join(val) for val in df.astype(str).values.tolist()]

index = 0
forDelete = []

for index in range(df.shape[0]):
    line = r[index]
    if mydict[line] < 5:
        #print(line, ">>", index)
        forDelete.append(index)
    index+=1
print(len(forDelete), " lines need to be deleted to meet all requirements")

3  lines need to be deleted to meet all requirements


In [11]:
if DELETE_RARE_ATTRIBUTES:
    df = df.drop(df.index[forDelete])

In [12]:
new_dict, new_k_anon = get_dict_and_k_anonimity()
print("After correction new k-anon = ", new_k_anon)

After correction new k-anon =  6


In [13]:
sorted_new_dict = sorted(new_dict.items(), key=lambda x:x[1])
for i in range(5):
    print(sorted_new_dict[i])

('***Beelineул. ВоронежскаяПрограммистHigh', 6)
('***Tele2ул. ВоронежскаяПрограммистVery high', 6)
('***Tele2Реки Смоленки наб.ПрограммистVery high', 7)
('***BeelineДмитровский пер.ПрограммистHigh', 8)
('***Beelineпр-кт ЗагородныйПрограммистVery high', 8)


In [14]:
path = filename.split('.')[0] + 'anonymized.xlsx'

if SAVE_ANON_FILE:
    df.to_excel(path, index = False)

In [15]:
df

,ФИО работника,Номер телефона,Адрес работы,Должность,Заработная плата
0,***,Megafon,ул. Чайковского,*,Low
1,***,Beeline,пр-кт Владимирский,*,Low
2,***,MTS,ул. Чайковского,*,Low
3,***,MTS,Павлоградский пер.,*,Low
4,***,Tele2,ул. Марата,*,Low
...,...,...,...,...,...
997490,***,MTS,ул. Камская,Инженер,Mid
997491,***,Beeline,Кузнечный пер.,Директор,High
997492,***,Megafon,2-я В.О. линия,Инженер,Low
997493,***,Megafon,ул. Маяковского,Вспомогательный персонал,Low
